# Notebook #2b: the spectral maps

This notebook creates the spectral maps. 

We calculate spectral maps and error maps; the latter is calculated with and without the systematic flux density error of 1%. 

These created .fits fits files are used in the PlottingImages notebook to re-create Figure 4.

The average values of the spectral index and its error (with and without the systematic uncertainty) are calculated using CARTA separately. For this, we use the region files contained in the REGIONS folder.

We take the following steps:

0) Loading packages
1) Create cutouts with updated WCS with the same centroid position for RACS mid and low
2) Create spectral maps
3) Repeat the spectral error maps with a systematic error

### 0) Loading packages:

In [1]:
path = '/Library/Fonts/Arial Unicode.ttf'

import numpy as np
import numpy.ma as ma

import aplpy

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord

import os
from astropy import units as u
from astropy.units import cds
from matplotlib import colors
cds.enable()  

# If the font should not be changed, comment out the next two lines:
prop = font_manager.FontProperties(fname=path)
mpl.rcParams['font.family'] = prop.get_name()
mpl.rcParams['pdf.fonttype']=42

params = {'text.usetex':False, 'mathtext.fontset':'custom', 'mathtext.default':'regular'}
mpl.rcParams.update(params)

single_col = 8.9 # cm
double_col = 18.3 # cm
def cm2inch(value):
    return value/2.54
FS = 22
LS = 22
MS = 14
MEW= 1.5

%matplotlib inline

from astropy.io import fits
from astropy import wcs
import FITS_tools
from astropy.wcs import WCS

### 1) Create cutouts with updated WCS with the same centroid position for RACS mid and low:

In [3]:
# The larger images:
RACS_MID = './FITS_IMAGES/RACS-MID1_1136-64_25arcsec.DROPDEG.fits'
RACS_LOW = './FITS_IMAGES/RACS-DR1_1135-62A.DROPDEG.fits'

In [4]:
# The central position:
centroid_position = SkyCoord('11h28m54.1820612352s -62d39m09.837096588s', frame='icrs')
cutout_size = u.Quantity((30., 30.), u.arcmin)

In [5]:
# Load the image and the WCS: RACS-LOW (UHF band)
filename = RACS_LOW

hdu = fits.open(filename)[0]
wcs = WCS(hdu.header)

# Make the cutout, including the WCS
cutout = Cutout2D(hdu.data, position=centroid_position, size=cutout_size, wcs=wcs)

# Put the cutout image in the FITS HDU
hdu.data = cutout.data

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header())

# Write the cutout to a new FITS file
cutout_filename = './FITS_IMAGES/RACS-LOW.cropped.fits'
hdu.writeto(cutout_filename, overwrite=True)

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [6]:
# Load the image and the WCS: RACS-MID (L band)
filename = RACS_MID

hdu = fits.open(filename)[0]
wcs = WCS(hdu.header)

# Make the cutout, including the WCS
cutout = Cutout2D(hdu.data, position=centroid_position, size=cutout_size, wcs=wcs)

# Put the cutout image in the FITS HDU
hdu.data = cutout.data

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header())

# Write the cutout to a new FITS file
cutout_filename = './FITS_IMAGES/RACS-MID.cropped.fits'
hdu.writeto(cutout_filename, overwrite=True)

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


### 2) Create spectral maps:

In [7]:
# The cropped files:
RACS_MID = './FITS_IMAGES/RACS-MID.cropped.fits'
RACS_LOW = './FITS_IMAGES/RACS-LOW.cropped.fits'

In [9]:
# Reading in the data: RACS-MID
fh = fits.open(RACS_MID)
data_MID = fh[0].data.squeeze() # drops the size-1 axes
header_MID = fh[0].header
mywcs_MID = WCS(header_MID).celestial

new_header_MID = FITS_tools.strip_headers.flatten_header(header_MID)
new_fh_MID = fits.PrimaryHDU(data=data_MID, header=new_header_MID)

fh.close()

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [10]:
# Reading in the data: RACS-LOW
fh = fits.open(RACS_LOW)
data_LOW = fh[0].data.squeeze() # drops the size-1 axes
header_LOW = fh[0].header
mywcs_LOW = WCS(header_LOW).celestial

new_header_LOW = FITS_tools.strip_headers.flatten_header(header_LOW)
new_fh_LOW = fits.PrimaryHDU(data=data_LOW, header=new_header_LOW)

fh.close()

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [14]:
# Defining the frequencies in GHz
nu_LOW = 0.8875
nu_MID = 1.3675

In [15]:
# RMS values in mJy/bm:
dflux_LOW = 0.5e-3
dflux_MID = 0.3e-3

In [16]:
# Calculating the spectral index and uncertainty for each pixel with sufficient flux in both bands:
alpha = np.zeros((len(data_LOW), len(data_LOW)), dtype='float32')
dalpha = np.zeros((len(data_LOW), len(data_LOW)), dtype='float32')
for i in range(len(data_LOW)):
    for j in range(len(data_LOW)):
        
        flux_LOW = data_LOW[i,j]

        # Note: this is a correction since the two datasets have the same beam size, but not the same pixel size.
        # So the pixel index needs to be re-scaled for the RACS-MID data.
        i_MID = int(i * len(data_MID) / len(data_LOW))
        j_MID = int(j * len(data_MID) / len(data_LOW))
        
        flux_MID = data_MID[i_MID, j_MID]

        # Flux thresholding:
        if flux_LOW > 2.5e-3 and flux_MID > 1e-3:
            
            alpha_ij = np.log10(flux_LOW / flux_MID) / np.log10(nu_LOW / nu_MID)
            dalpha_ij = abs((1. / np.log10(nu_LOW / nu_MID)) * np.sqrt((dflux_LOW/flux_LOW)**2. + (dflux_MID/flux_MID)**2.))
            
        else:
                
            alpha_ij = np.NaN
            dalpha_ij = np.NaN
            
        alpha[i,j] = alpha_ij       
        dalpha[i,j] = dalpha_ij          

In [17]:
# Saving alpha using the same header as the RACS data to use the correct coordinates:
fh = fits.open(RACS_LOW)
data_ALPHA = alpha
header_ALPHA = fh[0].header
mywcs_ALPHA = WCS(header_ALPHA).celestial

new_header_ALPHA = FITS_tools.strip_headers.flatten_header(header_ALPHA)
new_fh_ALPHA = fits.PrimaryHDU(data=data_ALPHA, header=new_header_ALPHA)

new_fh_ALPHA.writeto('./FITS_IMAGES/alpha.fits', overwrite=True)

fh.close()

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [18]:
# Saving alpha error: using the same header as the RACS data to use the correct coordinates:
fh = fits.open(RACS_LOW)
data_dALPHA = dalpha
header_dALPHA = fh[0].header
mywcs_dALPHA = WCS(header_dALPHA).celestial

new_header_dALPHA = FITS_tools.strip_headers.flatten_header(header_dALPHA)
new_fh_dALPHA = fits.PrimaryHDU(data=data_dALPHA, header=new_header_dALPHA)

new_fh_dALPHA.writeto('./FITS_IMAGES/dalpha.fits', overwrite=True)

fh.close()

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


### 3) Repeat the spectral error maps with a systematic error:

In [19]:
# Same as above: now introducing a 1% error in the calculation of the error:

alpha = np.zeros((len(data_LOW), len(data_LOW)), dtype='float32')
dalpha = np.zeros((len(data_LOW), len(data_LOW)), dtype='float32')
for i in range(len(data_LOW)):
    for j in range(len(data_LOW)):
        
        flux_LOW = data_LOW[i,j]

        # Note: this is a correction since the two datasets have the same beam size, but not the same pixel size.
        # So the pixel index needs to be re-scaled for the RACS-MID data.
        i_MID = int(i * len(data_MID) / len(data_LOW))
        j_MID = int(j * len(data_MID) / len(data_LOW))
        
        flux_MID = data_MID[i_MID, j_MID]

        # Flux thresholding:
        if flux_LOW > 2.5e-3 and flux_MID > 1e-3:

            # Adding the systematic uncertainty:
            dflux_LOW_ij = (dflux_LOW**2 + 0.01*flux_LOW**2)**0.5
            dflux_MID_ij = (dflux_MID**2 + 0.01*flux_MID**2)**0.5
            
            alpha_ij = np.log10(flux_LOW / flux_MID) / np.log10(nu_LOW / nu_MID)
            dalpha_ij = abs((1. / np.log10(nu_LOW / nu_MID)) * np.sqrt((dflux_LOW_ij/flux_LOW)**2. + (dflux_MID_ij/flux_MID)**2.))
            
        else:
                
            alpha_ij = np.NaN
            dalpha_ij = np.NaN
            
        alpha[i,j] = alpha_ij       
        dalpha[i,j] = dalpha_ij          

In [20]:
# Saving alpha error with systematic uncertainty: using the same header as the RACS data to use the correct coordinates:

fh = fits.open(RACS_LOW)
data_dALPHA = dalpha
header_dALPHA = fh[0].header
mywcs_dALPHA = WCS(header_dALPHA).celestial

new_header_dALPHA = FITS_tools.strip_headers.flatten_header(header_dALPHA)
new_fh_dALPHA = fits.PrimaryHDU(data=data_dALPHA, header=new_header_dALPHA)

new_fh_dALPHA.writeto('./FITS_IMAGES/dalpha_with_syst.fits', overwrite=True)

fh.close()

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
